# Assignment

## About Dataset:

The dataset is frome **Kaggle**(https://www.kaggle.com/datasets/jessemostipak/hotel-booking-demand).

## Install:

In [2]:
!pip install pyspark

  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845518 sha256=91713f5be2c8981469506de6d6d61e39c77062ec02d87584b16912cdcd056a81
  Stored in directory: c:\users\acer\appdata\local\pip\cache\wheels\43\dc\11\ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


## Import Libraries:

In [3]:
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import pyspark
from pyspark.sql import functions as F
from pyspark.sql import types

from pyspark.ml.feature import Imputer, VectorAssembler, StringIndexer
from pyspark.ml.regression import RandomForestRegressor, DecisionTreeRegressor, GBTRegressor, LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

warnings.filterwarnings('ignore')

## Connect to the Spark server:

In [4]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

## Obtain the Data:

In [5]:
fullpath = 'hotel_bookings.csv'

data = spark.read.csv(fullpath)

In [6]:
data

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string, _c23: string, _c24: string, _c25: string, _c26: string, _c27: string, _c28: string, _c29: string, _c30: string, _c31: string]

In [7]:
# read.csv is very similar to the Pandas version
data = spark.read.csv(fullpath,
                     sep=',',
                     inferSchema=True,
                     header=True,
                     multiLine=True)

data.printSchema()

root
 |-- hotel: string (nullable = true)
 |-- is_canceled: integer (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_date_year: integer (nullable = true)
 |-- arrival_date_month: string (nullable = true)
 |-- arrival_date_week_number: integer (nullable = true)
 |-- arrival_date_day_of_month: integer (nullable = true)
 |-- stays_in_weekend_nights: integer (nullable = true)
 |-- stays_in_week_nights: integer (nullable = true)
 |-- adults: integer (nullable = true)
 |-- children: string (nullable = true)
 |-- babies: integer (nullable = true)
 |-- meal: string (nullable = true)
 |-- country: string (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- distribution_channel: string (nullable = true)
 |-- is_repeated_guest: integer (nullable = true)
 |-- previous_cancellations: integer (nullable = true)
 |-- previous_bookings_not_canceled: integer (nullable = true)
 |-- reserved_room_type: string (nullable = true)
 |-- assigned_room_type: string (nullab

## Data Cleaning:

### Drop unneeded columns:

In [8]:
# these columns are useless to us, drop them
drop_cols = ['agent', 'company']

data = data.drop(*drop_cols)

### Check for duplicates:

In [9]:
print('Count of rows: {0}'.format(data.count()))
print('Count of distinct rows: {0}'.format(data.distinct().count()))

Count of rows: 119390
Count of distinct rows: 87370


In [14]:
# Check for missing values
data.select(*[
    (
        F.count(F.when((F.isnan(c) | F.col(c).isNull()), c)) if t not in ("timestamp", "date")
        else F.count(F.when(F.col(c).isNull(), c))
    ).alias(c)
    for c, t in data.dtypes if c in data.columns
    ]).show()

+-----+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+----+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+--------------------+-------------+---+---------------------------+-------------------------+------------------+-----------------------+
|hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|days_in_waiting_list|customer_type|adr|required_car_parking_spaces|total_of_special_requests|reservation_status|reser

In [15]:
# Drop the null values from humidity and barometer columns
data.na.drop(how="any").show(truncate=False)

+------------+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+----+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+--------------------+-------------+------+---------------------------+-------------------------+------------------+-----------------------+
|hotel       |is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|days_in_waiting_list|customer_type|adr   |required_car_parking_spaces|total_of_special_requests|rese

In [19]:
# Latitude should be between (-90, 90), otherwise set null
data = data.withColumn('is_canceled', F.when(F.col('is_canceled').rlike('^-?\d+\.?\d+$'), F.col('is_canceled')))
data = data.withColumn('is_canceled', F.col('is_canceled').cast(types.FloatType()))
data = data.withColumn('is_canceled', F.when((-90 <= F.col('is_canceled')) & (F.col('is_canceled') <= 90), F.col('is_canceled')))

In [20]:
# Longitude should be between (-180, 180), otherwise set null
data = data.withColumn('is_repeated_guest', F.when(F.col('is_repeated_guest').rlike('^-?\d+\.?\d+$'), F.col('is_repeated_guest')))
data = data.withColumn('is_repeated_guest', F.col('is_repeated_guest').cast(types.FloatType()))
data = data.withColumn('is_repeated_guest', F.when((-180 <= F.col('is_repeated_guest')) & (F.col('is_repeated_guest') <= 180), F.col('is_repeated_guest')))

In [21]:
# Posting date should be in format yyyy-MM-dd HH:mm:ss.SSSS
data = data.withColumn('arrival_date_year', F.when(F.col('arrival_date_year').rlike('^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}-\d{4}$'), F.col('arrival_date_year')))
data = data.withColumn('arrival_date_year', F.col('arrival_date_year').cast(types.TimestampType()))

### Descriptive Statistics:

In [22]:
data.select('is_canceled','is_repeated_guest','arrival_date_year').summary().show()

+-------+-----------+-----------------+
|summary|is_canceled|is_repeated_guest|
+-------+-----------+-----------------+
|  count|          0|                0|
|   mean|       null|             null|
| stddev|       null|             null|
|    min|       null|             null|
|    25%|       null|             null|
|    50%|       null|             null|
|    75%|       null|             null|
|    max|       null|             null|
+-------+-----------+-----------------+



### Data Wrangling:

In [23]:
# both columns starts with 0, log(0) is undefined so we should put +1
data = data.withColumn('is_canceled', F.log10(F.col('is_canceled') + 1))
data = data.withColumn('is_repeated_guest', F.log10(F.col('is_repeated_guest') + 1))

In [24]:
data.select('is_canceled','is_repeated_guest','arrival_date_year').summary().show()

+-------+-----------+-----------------+
|summary|is_canceled|is_repeated_guest|
+-------+-----------+-----------------+
|  count|          0|                0|
|   mean|       null|             null|
| stddev|       null|             null|
|    min|       null|             null|
|    25%|       null|             null|
|    50%|       null|             null|
|    75%|       null|             null|
|    max|       null|             null|
+-------+-----------+-----------------+



### Data Exploration:

In [27]:
data.registerTempTable('data.hotel')

state_counts = spark.sql(r"""SELECT state, COUNT(state) AS total 
                                     FROM data.hotel 
                                     GROUP BY state 
                                     ORDER BY total desc """)
state_counts.show()

Py4JJavaError: An error occurred while calling o23.sql.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:6

## Histogram:

In [29]:
#top three manufacturers
df_result = spark.sql('SELECT manufacturer, price FROM data WHERE manufacturer IN ("is_canceled", "is_repeated_guest", "arrival_date_year")')

#convert to pandas prior to visualization
df_result = df_result.toPandas()

AnalysisException: Column 'manufacturer' does not exist. Did you mean one of the following? [data.adr, data.adults, data.country, data.hotel, data.babies, data.children, data.meal, data.lead_time, data.is_canceled, data.customer_type, data.deposit_type, data.market_segment, data.arrival_date_year, data.booking_changes, data.assigned_room_type, data.is_repeated_guest, data.reserved_room_type, data.arrival_date_month, data.reservation_status, data.days_in_waiting_list, data.distribution_channel, data.previous_cancellations, data.stays_in_week_nights, data.arrival_date_week_number, data.reservation_status_date, data.stays_in_weekend_nights, data.total_of_special_requests, data.arrival_date_day_of_month, data.required_car_parking_spaces, data.previous_bookings_not_canceled]; line 1 pos 43;
'Project ['manufacturer, 'price]
+- 'Filter 'manufacturer IN (is_canceled,is_repeated_guest,arrival_date_year)
   +- SubqueryAlias data
      +- View (`data`, [hotel#81,is_canceled#1422,lead_time#83,arri

In [30]:
sns.set_context("notebook", font_scale=1.25)
sns.histplot(data=df_result, x='is_canceled', hue='is_repeated_guest', binwidth=0.3)

NameError: name 'df_result' is not defined